# BUFN400 Self Study Problems: Time Value of Money
## By James Zhang

In [2]:
import pandas as pd
import numpy as np
import scipy
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

# Problem 1
In one short sentence, why is the largest representable 32-bit integer equal to $2^{32} - 1$ and not $2^{32}$?

**Solution:** We start counting from 0, and not 1. Consider a 4-bit integer. The maximum 4-bit integer is $1111_2 = 15_{10}$. Thus, for this example, it is $2^{4} - 1$. For a 32-bit integer, it is the same, and this the largerst integer is $2^{32} -1$. 

# Problem 2
1. Study the following example to make sure that you understand Pandas syntax for creating simple dataframes. This example does not use np.datetime64 and np.timedelta64 to keep track of time.

In [16]:
days_in_year = 360
maturity_in_days = 180
rate = 0.06
v0 = 1000.00

#maturity_in_years = float(maturity_in_days) / float(days_in_year)  # casting not needed? check to make sure!
maturity_in_years = maturity_in_days / days_in_year

df = pd.DataFrame({'type' : ['simple interest'] + ['compound interest'] * 5 + ['discount rate'],
                   'freq' : ['N/A'] + [1, 2, 12, days_in_year, np.inf] + ['N/A'], # compounding frequency
                   'notation' : ['r0', 'r1', 'r2', 'r12', 'r' + str(days_in_year), 'rinf', 'rd'] ,
                   'days_in_year' : [days_in_year] * 7,
                   'maturity' : [maturity_in_days] * 7,
                   'rate' : [rate] * 7,
                   'maturity_in_days' : [maturity_in_days] * 7,
                   'maturity_in_years' :  [maturity_in_years] * 7,
                   'pv' : [v0] * 7,  # "present value", like v_0
                   'fv' : [np.nan] * 7})  # "future value", like v_1


df.loc[0, 'fv'] = df.loc[0, 'pv'] *  (1.00 + df.loc[0, 'rate'] * df.loc[0, 'maturity_in_years'])
df

,type,freq,notation,days_in_year,maturity,rate,maturity_in_days,maturity_in_years,pv,fv
0,simple interest,N/A,r0,360,180,0.06,180,0.5,1000.0,1030.0
1,compound interest,1,r1,360,180,0.06,180,0.5,1000.0,NaN
2,compound interest,2,r2,360,180,0.06,180,0.5,1000.0,NaN
3,compound interest,12,r12,360,180,0.06,180,0.5,1000.0,NaN
4,compound interest,360,r360,360,180,0.06,180,0.5,1000.0,NaN
5,compound interest,inf,rinf,360,180,0.06,180,0.5,1000.0,NaN
6,discount rate,N/A,rd,360,180,0.06,180,0.5,1000.0,NaN


2. Use time-value-of-money formulas to change the missing values (np.nan) to their correct values.

In [20]:
# Compound interest cases, rows 1-4
for i in range(1, 5):
    v0 = df["pv"][i]
    m = df["maturity_in_years"][i]
    N = df["freq"][i]
    rN = df["rate"][i]
    df["fv"][i] = v0 * ((1 + rN / N) ** (N * m))

# Continuous interest case, row 5
df["fv"][5] = df["pv"][5] * (np.exp(df["rate"][5] * df["maturity_in_years"][5]))

# Discount rate, row 6
df["fv"][6] = df["pv"][6] / (1 - df["rate"][6] * df["maturity_in_years"][6])
df

,type,freq,notation,days_in_year,maturity,rate,maturity_in_days,maturity_in_years,pv,fv
0,simple interest,N/A,r0,360,180,0.06,180,0.5,1000.0,1030.000000
1,compound interest,1,r1,360,180,0.06,180,0.5,1000.0,1029.563014
2,compound interest,2,r2,360,180,0.06,180,0.5,1000.0,1030.000000
3,compound interest,12,r12,360,180,0.06,180,0.5,1000.0,1030.377509
4,compound interest,360,r360,360,180,0.06,180,0.5,1000.0,1030.451958
5,compound interest,inf,rinf,360,180,0.06,180,0.5,1000.0,1030.454534
6,discount rate,N/A,rd,360,180,0.06,180,0.5,1000.0,1030.927835


3. Determine by experimentation whether the values in the column 'fv' increase monotonically for any constant values for 'days_in_year', 'maturity_in_days', and 'rate'.

Yes, all of these changes result in a monotonic increase, and this intuitively makes sense.

4. Note that maturity_in_years is obtained by dividing two integers. Verify what happens in Python and numpy when an integer is divided by an integer and "true division" would yield a result with a fraction but integer division (floor division) would yield an integer result? For example, do we obtain 
$1 / 2
=
0.50
\text{ or }
1
/
2
=
0$

In [24]:
f"Numpy: {np.int(1) / np.int(2)} and Python: {1/2}"

'Numpy: 0.5 and Python: 0.5'

# Problem 3

1. Write out by hand the algebraic expression for the compound interest formula for 
$$v_1 = v_0(1 + r / N)^{Nm}$$
for $m = 1$ year and $N = 1, 2, 3, 4$ compounding periods.

**Solution:** 
$$v_1 = v_0(1 + r), \ v_1 = v_0(1 + r / 2)^{2}, \ v_1 = v_0(1 + r / 3)^3, \ v_1 = v_0(1 + r / 4)^4$$

Yes you can interpret the terms as "interest on interest" and etc, and this is similar in the continuous compounding case where instead of a summation you take the limit.

# Problem 4

Suppose I am buying 91-day commercial paper in blocks of 10 million dollars. The discount rate (not discount factor) I should quote for 91-day paper is 3.02 percent, and the discount rate I should quote for 84-day commercial paper is 3.01 percent. I make the mistake of incorrectly quoting the discount rate for 84-day paper when I am actually buying 91-day paper.

Am I overpaying or underpaying? By how many dollars do I overpay or underpay?
Now suppose I quote a discount factor (times 10 million dollars) for the price of 84-day paper when I am actually buying 91-day paper.

Am I overpaying or underpaying? By how many dollars do I overpay or underpay?

What issue does this problem illustrate?

#### Incorrect Discount Rate for 91-day Paper
Given:
- Correct discount rate for 91-day paper: 3.02%
- Incorrectly quoted discount rate for 84-day paper: 3.01%

**Calculation:**
- Calculate the difference in rates: $3.02\% - 3.01\% = 0.01\%$
- Convert this difference to an annualized rate: $0.01\% \times \frac{91}{84}$

Now, use the formula for discount rates to find the difference in price:

$$\text{Difference in price} = \text{Amount} \times \text{Days} \times (\text{Correct rate} - \text{Incorrect rate})$$

#### Incorrect Discount Factor for 91-day Paper
Given:
- Discount factor for 84-day paper (instead of 91-day paper)

**Calculation:**
- Use the correct formula to find the difference in price due to the incorrect discount factor.

$$\text{Difference in price} = \text{Amount} \times (\text{Correct factor} - \text{Incorrect factor})$$

- For the incorrect discount rate scenario, the difference could result in either overpayment or underpayment depending on the comparison of rates.
- For the incorrect discount factor scenario, the difference will likely result in overpayment or underpayment.
This problem highlights the consequences of using incorrect financial parameters (rates or factors) for different time periods when purchasing commercial paper. It demonstrates the impact of misquoting rates or factors on the price paid, potentially leading to financial losses or gains.


# Problem 5
Consider the quadratic function $f(r) = \frac{1}{2}cr^2 + br + a$
Can you explain some intuitive logic for how to map the parameters $a, b, c$ into the concepts level, slope, and curvature? (There might be more than one way to do this.)

- Curvature: the greater the $c$ value, the less curvy and more sharp (narrower) the quadratic is.
- Slope: the greater the $b$ value, the greater the overall slope of the quadratic.
- Level: the $a$ value changes the y-intercept, thus changing the "level" of the quadratic.

# Problem 6

In [4]:
nobs = 10**5  # Number of observations
# Define a random number generator using numpy recommendation:
rng = np.random.default_rng()
# Generate random integers between 1 and 30 (years):
m = rng.integers(1, 30, nobs) + 0.01 * rng.standard_normal(nobs)
# Generate normally distributed random numbers (interest rates) with desired mean and variance:
r = (0.50 * rng.standard_normal(nobs) + 3.00) * 0.01

N = 2

# Calculate discount factors:
res0 = np.exp(-r * m)
res1 = 1 / ((1 + r / N)**(m * N))
res2 = np.exp(-np.log(1 + r / N) * (m * N))

print(f"{res0=}\n{res1=}\n{res2=}\n")

# Time speed of discount factor calculations:
print("\nTime to calculate simple interest:")
%timeit -r 7 -n 10 ress = 1 / (1 + r * m)
print("\nTime to calculate continuously compounded interest with exp function:")
%timeit -r 7 -n 10 res0 = np.exp(-r * m)
print("\nTime to calculate compound interest:")
%timeit -r 7 -n 10 res1 = (1 + r / N)**(-m * N)
print("\nTime to calculate compound interest using exp and log functions instead of power function:")
%timeit -r 7 -n 10 res2 = np.exp(-np.log(1 + r / N) * (m * N))

print(f"\nContinuously compounded interest is different from compound interest for {N} periods: \n{np.isclose(res1, res0).all()=}")

print(f"\nCalculating compound interest with exp and log")

res0=array([0.79747807, 0.5209257 , 0.84364669, ..., 0.46483714, 0.97127529,
       0.76817985])
res1=array([0.79892414, 0.52312604, 0.84465405, ..., 0.46744137, 0.97147722,
       0.7695065 ])
res2=array([0.79892414, 0.52312604, 0.84465405, ..., 0.46744137, 0.97147722,
       0.7695065 ])


Time to calculate simple interest:
125 µs ± 38 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Time to calculate continuously compounded interest with exp function:
572 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Time to calculate compound interest:
1.8 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Time to calculate compound interest using exp and log functions instead of power function:
1.11 ms ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Continuously compounded interest is different from compound interest for 2 periods: 
np.isclose(res1, res0).all()=False

Calculating compound interest with exp and log
